In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import os
import seaborn as sns
import statsmodels.formula.api as sm
from numpy.polynomial.polynomial import polyfit
from sklearn.model_selection import train_test_split
import sklearn
from math import sqrt
import glob

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [3]:
##Remote sensing data:
import glob

path = r'C:/Users/arj26323//Documents/All extract/GAVA_combined'

all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    li.append(df1)

df1 = pd.concat(li, axis=0, ignore_index=True)

df1

display(df1)

# for col in df1.columns:
#     print(col)

,Latitude,Longitude,Plant_Biomass,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,ST_TRAD,ST_URAD,Site,Year,date,flats,index,SR_B6,SR_QA_AEROSOL,ST_B10
0,31.308449,-81.415129,440.816,5440,0,319.0,0.032183,0.05025,0.04024,0.176805,...,8769,3161,GA,2013,1984-05-14,0.000630,0,NaN,NaN,NaN
1,31.308449,-81.415129,468.640,5440,0,319.0,0.032183,0.05025,0.04024,0.176805,...,8769,3161,GA,2015,1984-05-14,0.000630,1,NaN,NaN,NaN
2,31.308449,-81.415129,170.976,5440,0,319.0,0.032183,0.05025,0.04024,0.176805,...,8769,3161,GA,2016,1984-05-14,0.000630,2,NaN,NaN,NaN
3,31.308449,-81.415129,140.192,5440,0,319.0,0.032183,0.05025,0.04024,0.176805,...,8769,3161,GA,2018,1984-05-14,0.000630,3,NaN,NaN,NaN
4,31.308449,-81.415129,435.200,5440,0,319.0,0.032183,0.05025,0.04024,0.176805,...,8769,3161,GA,2021,1984-05-14,0.000630,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778649,31.351196,-81.333574,340.736,21824,0,NaN,0.025747,0.02847,0.03705,0.038837,...,7352,574,GA,2004,2024-12-21,0.001119,625,0.106102,96.0,285.266203
1778650,31.351196,-81.333574,695.056,21824,0,NaN,0.025747,0.02847,0.03705,0.038837,...,7352,574,GA,2005,2024-12-21,0.001119,626,0.106102,96.0,285.266203
1778651,31.351196,-81.333574,575.312,21824,0,NaN,0.025747,0.02847,0.03705,0.038837,...,7352,574,GA,2006,2024-12-21,0.001119,627,0.106102,96.0,285.266203
1778652,31.351196,-81.333574,374.208,21824,0,NaN,0.025747,0.02847,0.03705,0.038837,...,7352,574,GA,2020,2024-12-21,0.001119,628,0.106102,96.0,285.266203


In [4]:
##No way to remove sites 8 and 9 without Zone/Transect info

In [5]:
#Distinguishing between sensors, in order to make indices: USE PRESENCE OF DATA IN B10

df1['Sensor'] = np.where(df1['ST_B10'].isna(),'Landsat 5/7', 'Landsat 8/9')

df1['Date'] = pd.to_datetime(df1['Year'], format='%Y')

#Datetime and year for LS data and biomass sampling data
df1['LS_datetime'] = pd.to_datetime(df1['date'], format='%Y-%m-%d')
df1['LS_year'] = df1['LS_datetime'].dt.strftime('%Y')

df1['Sampling_datetime'] = pd.to_datetime(df1['Date'])
df1['Sampling_year'] = df1['Sampling_datetime'].dt.strftime('%Y')

df1['Month'] = df1['LS_datetime'].dt.month #IMPORTANT FOR VARIABLES! MUST BE SAMPLING LS TIME

# df1[df1['Sensor'] == 'Landsat 5/7'] #652450 5/7, 1126204 8/9

# df1['Month']

In [6]:
#Renaming bands, based on Landsat sensors

df1['Blue_band'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['SR_B2'], df1['SR_B1'])
df1['Green_band'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['SR_B3'], df1['SR_B2'])
df1['Red_band'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['SR_B4'], df1['SR_B3'])
df1['NIR_band'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['SR_B5'], df1['SR_B4'])
df1['SWIR1_band'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['SR_B6'], df1['SR_B5'])
df1['SWIR2_band'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['SR_B7'], df1['SR_B7'])

#THERMAL BANDS
df1['thermal_band_kelvin'] = np.where(df1['Sensor'] == 'Landsat 8/9', df1['ST_B10'], df1['ST_B6'])
df1['thermal_band_celsius'] = df1['thermal_band_kelvin'] - 273.15

##Indices (make sure calculations are accurate):
df1['savi'] = ((df1['NIR_band']-df1['Red_band'])*1.5)/(df1['NIR_band']+df1['Red_band']+0.5)
df1['wdrvi5'] = (0.5*df1['NIR_band']-df1['Red_band'])/(0.5*df1['NIR_band']+df1['Red_band'])
df1['nd_r_g'] = (df1['Red_band']-df1['Green_band'])/(df1['Red_band']+df1['Green_band'])
df1['nd_g_b'] = (df1['Green_band']-df1['Blue_band'])/(df1['Green_band']+df1['Blue_band'])
df1['nd_swir2_nir'] = (df1['SWIR2_band']-df1['NIR_band'])/(df1['SWIR2_band']+df1['NIR_band'])
df1['nd_swir2_r'] = (df1['SWIR2_band']-df1['Red_band'])/(df1['SWIR2_band']+df1['Red_band'])
df1['evi'] =((df1['NIR_band']-df1['Red_band'])*2.5)/(df1['NIR_band']+6*df1['Red_band']+7.5*df1['Blue_band']+1)
df1['ndvi'] = ((df1['NIR_band']-df1['Red_band']))/(df1['NIR_band']+df1['Red_band'])

In [7]:
# df1['date'] is the Landsat extraction date, ranging from 1985-2024! df1['Date'] is the sampling date for each site! (2000-2?)
#df1['Year'] is the sampling year as well! Starting in 2000
# # df1['Year']
# for col in df1.columns:
#     print(col)

# df1['thermal_band_celcius']

In [8]:
pairsite = df1[['Latitude', 'Longitude', 'Site']].drop_duplicates()
# pairsite 

In [9]:
##ADDITIONAL GROUP BY LS YEAR

def seasonal_band_stats(df, start_month, end_month, bands, stat_name='mean'):
    # Filter by month range
    df_season = df[(df['Month'] >= start_month) & (df['Month'] <= end_month)]
    
    # Filter to only those rows where LS_year matches Sampling_year
    df_season = df_season[df_season['LS_year'] == df_season['Sampling_year']]

    # Grouping by location and both year types
    group_cols = ['Latitude', 'Longitude', 'Sampling_year', 'LS_year']  ##ADD SITE???????
    
    # Choose aggregation function
    if stat_name == 'mean':
        agg_func = 'mean'
    elif stat_name == 'min':
        agg_func = 'min'
    elif stat_name == 'max':
        agg_func = 'max'
    elif stat_name == 'median':
        agg_func = 'median'
    elif stat_name == 'sum':
        agg_func = 'sum'    
    else:
        raise ValueError("stat_name must be 'mean', 'min', 'max', 'sum,' or 'median'")
    
    # Perform aggregation
    df_grouped = df_season.groupby(group_cols)[bands].agg(agg_func).reset_index()
    
    # Rename columns to reflect the operation and month window
    suffix = f'{stat_name}_{start_month:02d}_{end_month:02d}'
    df_grouped = df_grouped.rename(columns={col: f'{col}_{suffix}' for col in bands})
    
    return df_grouped

In [10]:
##Flooding dataset                                              )

df1_flooding = df1.groupby(['Latitude', 'Longitude', 'Plant_Biomass','Sampling_year']).apply(
    lambda g: (g['flats'] >= 0.2).mean()).reset_index(name='Percent_flooding'
                                                     )
##DO NOT GROUP FLOODING PERCENT BY LS_YEAR - you want the total percent flooding over the entire time period. ADD AT END?

df1_flooding

,Latitude,Longitude,Plant_Biomass,Sampling_year,Percent_flooding
0,31.308449,-81.415129,71.520,2023,0.014085
1,31.308449,-81.415129,140.192,2018,0.014085
2,31.308449,-81.415129,161.872,2020,0.014085
3,31.308449,-81.415129,170.976,2016,0.014085
4,31.308449,-81.415129,250.240,2019,0.014085
...,...,...,...,...,...
4055,37.482109,-75.818885,226.400,1999,0.050000
4056,37.482109,-75.818885,771.200,2000,0.050000
4057,37.482293,-75.819265,104.000,1999,0.133080
4058,37.482293,-75.819265,156.000,2002,0.133080


In [43]:
# max(df1_flooding['Percent_flooding'])

0.9128787878787878

In [11]:
##FLATS+ FILTERING!!
df1_filtered = df1[df1['flats'] < 0.2]

In [12]:
##Calculating variables of interest

bands_of_interest = ['ndvi', 'evi', 'nd_swir2_r', 'nd_swir2_nir', 'nd_g_b', 'nd_r_g', 'wdrvi5', 'savi', 'SWIR2_band', \
                     'SWIR1_band', 'NIR_band', 'Red_band', 'Green_band', 'Blue_band', 'flats', 'thermal_band_celsius']

##AUGOCT - PEAK
# Calculate average
mean_augoct = seasonal_band_stats(df1_filtered, 8, 10, bands_of_interest, stat_name='mean')
# Calculate max
max_augoct = seasonal_band_stats(df1_filtered, 8, 10, bands_of_interest, stat_name='max')
# Calculate min
min_augoct = seasonal_band_stats(df1_filtered, 8, 10, bands_of_interest, stat_name='min')
# Calculate median
med_augoct = seasonal_band_stats(df1_filtered, 8, 10, bands_of_interest, stat_name='median')
# Calculate SUM
sum_augoct = seasonal_band_stats(df1_filtered, 8, 10, bands_of_interest, stat_name='sum')

##MAYOCT - GROWING
# Calculate average
mean_mayoct = seasonal_band_stats(df1_filtered, 5, 10, bands_of_interest, stat_name='mean')
# Calculate max
max_mayoct = seasonal_band_stats(df1_filtered, 5, 10, bands_of_interest, stat_name='max')
# Calculate min
min_mayoct = seasonal_band_stats(df1_filtered, 5, 10, bands_of_interest, stat_name='min')
# Calculate median
med_mayoct = seasonal_band_stats(df1_filtered, 5, 10, bands_of_interest, stat_name='median')
# Calculate SUM
sum_mayoct = seasonal_band_stats(df1_filtered, 5, 10, bands_of_interest, stat_name='sum')

##MARMAY - EARLY
# Calculate average
mean_marmay = seasonal_band_stats(df1_filtered, 3, 5, bands_of_interest, stat_name='mean')
# Calculate max
max_marmay = seasonal_band_stats(df1_filtered, 3, 5, bands_of_interest, stat_name='max')
# Calculate min
min_marmay = seasonal_band_stats(df1_filtered, 3, 5, bands_of_interest, stat_name='min')
# Calculate median
med_marmay = seasonal_band_stats(df1_filtered, 3, 5, bands_of_interest, stat_name='median')
# Calculate SUM
sum_marmay = seasonal_band_stats(df1_filtered, 3, 5, bands_of_interest, stat_name='sum')

##JANDEC - YEAR
# Calculate average
mean_year = seasonal_band_stats(df1_filtered, 1, 12, bands_of_interest, stat_name='mean')
# Calculate max
max_year = seasonal_band_stats(df1_filtered, 1, 12, bands_of_interest, stat_name='max')
# Calculate min
min_year = seasonal_band_stats(df1_filtered, 1, 12, bands_of_interest, stat_name='min')
# Calculate median
med_year = seasonal_band_stats(df1_filtered, 1, 12, bands_of_interest, stat_name='median')
# Calculate SUM
sum_year = seasonal_band_stats(df1_filtered, 1, 12, bands_of_interest, stat_name='sum')

In [13]:
df_final = mean_augoct
df_final = df_final.merge(max_augoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(min_augoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(med_augoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(sum_augoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')

df_final = df_final.merge(mean_mayoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(max_mayoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(min_mayoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(med_mayoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(sum_mayoct, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')

df_final = df_final.merge(mean_marmay, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(max_marmay, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(min_marmay, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(med_marmay, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(sum_marmay, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')

df_final = df_final.merge(mean_year, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(max_year, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(min_year, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(med_year, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')
df_final = df_final.merge(sum_year, on=['Latitude', 'Longitude', 'Sampling_year', 'LS_year'], how='left')

#Add df1_flooding
df_final = df_final.merge(df1_flooding, on=['Latitude', 'Longitude', 'Sampling_year'], how='left')

# #Add elevation
# elev_10 = pd.read_csv('C:/Users/arj26323/Documents/All extract/Elevation/elev.csv')
# elev_10['Sampling_datetime'] = pd.to_datetime(elev_10['Date'])
# elev_10['Sampling_year'] = elev_10['Sampling_datetime'].dt.strftime('%Y')
# elev_10 = elev_10[['Sampling_year', 'Latitude', 'Longitude', 'elevation']]

##CONVERT TO Z* (Z-star = (Elev. - MSL)/(MHHW-MSL)). ALL RELATIVE TO NAVD88
##MHHW for GA (Fort Pulaski): 1.052
##MHHW for VA (Wachapreague, VA - Station ID: 8631044): 0.569
##MSL for GA: -0.071
##MSL for VA: -0.106



# #Add slope
# slope_10 = pd.read_csv('C:/Users/arj26323/Documents/All extract/Elevation/slope.csv')
# slope_10['Sampling_datetime'] = pd.to_datetime(slope_10['Date'])
# slope_10['Sampling_year'] = slope_10['Sampling_datetime'].dt.strftime('%Y')
# slope_10 = slope_10[['Sampling_year', 'Latitude', 'Longitude', 'slope']]

# #Add DSM
# dsm = pd.read_csv('C:/Users/arj26323/Documents/All extract/Elevation/dsm.csv')
# dsm['Sampling_datetime'] = pd.to_datetime(dsm['Date'])
# dsm['Sampling_year'] = dsm['Sampling_datetime'].dt.strftime('%Y')
# dsm = dsm[['Sampling_year', 'Latitude', 'Longitude', 'DSM']]

# df_final = df_final.merge(elev_10, on=['Latitude', 'Longitude', 'Sampling_year'], how='left')
# df_final = df_final.merge(dsm, on=['Latitude', 'Longitude', 'Sampling_year'], how='left')
# df_final = df_final.merge(slope_10, on=['Latitude', 'Longitude', 'Sampling_year'], how='left')

# df_final['DSM'] = df_final.groupby(['Latitude', 'Longitude'])['DSM'].transform('first') ##Populating empty DSM vals

# #Adding site information back
# df_final = df_final.merge(pairsite, on=['Latitude', 'Longitude'], how='left')

#Add USGS river discharge data (BELOW)

df_final #elev merge added 6 extra rows...

,Latitude,Longitude,Sampling_year,LS_year,ndvi_mean_08_10,evi_mean_08_10,nd_swir2_r_mean_08_10,nd_swir2_nir_mean_08_10,nd_g_b_mean_08_10,nd_r_g_mean_08_10,...,SWIR2_band_sum_01_12,SWIR1_band_sum_01_12,NIR_band_sum_01_12,Red_band_sum_01_12,Green_band_sum_01_12,Blue_band_sum_01_12,flats_sum_01_12,thermal_band_celsius_sum_01_12,Plant_Biomass,Percent_flooding
0,31.308449,-81.415129,2013,2013,0.608544,0.185087,0.262307,-0.411777,0.188730,0.020280,...,1.029547,1.873330,2.255250,0.615370,0.582122,0.407772,0.013652,401.087648,440.816,0.014085
1,31.308449,-81.415129,2014,2014,0.439015,0.130858,0.239136,-0.226656,0.110968,0.055654,...,3.258605,5.330702,5.221170,2.169990,1.989837,1.653485,0.157165,836.528459,342.656,0.014085
2,31.308449,-81.415129,2015,2015,0.502339,0.148550,0.240085,-0.298153,0.132298,0.062364,...,2.705847,4.555525,4.610910,1.622677,1.479897,1.100755,0.050575,791.568502,468.640,0.014085
3,31.308449,-81.415129,2016,2016,0.548571,0.179854,0.294428,-0.302436,0.142160,0.049064,...,2.989195,5.151740,5.522880,1.723452,1.580947,1.177742,0.053336,946.218879,170.976,0.014085
4,31.308449,-81.415129,2017,2017,0.569564,0.185837,0.290756,-0.334508,0.110515,0.036491,...,3.623960,6.163502,6.167957,1.982347,1.833710,1.470187,0.026577,1069.804259,438.688,0.014085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552,37.481998,-75.819360,2002,2002,0.407277,0.103597,-0.040103,-0.441163,0.135267,-0.003203,...,0.234530,0.471497,0.670103,0.289227,0.302592,0.211952,0.104006,131.053753,317.680,0.109272
3553,37.482109,-75.818885,1999,1999,0.568293,0.130879,0.100846,-0.495865,0.226073,-0.074702,...,0.034822,0.076925,0.103325,0.028442,0.033035,0.020852,0.000909,15.158241,226.400,0.050000
3554,37.482109,-75.818885,2000,2000,0.564525,0.160160,0.128746,-0.473543,0.189670,-0.008712,...,0.273442,0.539037,0.617990,0.242532,0.222513,0.177935,0.135329,87.070672,771.200,0.050000
3555,37.482293,-75.819265,1999,1999,0.519408,0.130932,0.150791,-0.399922,0.266252,-0.037644,...,0.192392,0.380602,0.453697,0.177405,0.182520,0.138795,0.027743,73.847030,104.000,0.133080


In [58]:
# df_final[df_final['Latitude'].astype(str).str.startswith('37')]
##Roughly 500 VA, 3000 GA

,Latitude,Longitude,Sampling_year,LS_year,ndvi_mean_08_10,evi_mean_08_10,nd_swir2_r_mean_08_10,nd_swir2_nir_mean_08_10,nd_g_b_mean_08_10,nd_r_g_mean_08_10,...,SWIR2_band_sum_01_12,SWIR1_band_sum_01_12,NIR_band_sum_01_12,Red_band_sum_01_12,Green_band_sum_01_12,Blue_band_sum_01_12,flats_sum_01_12,thermal_band_celsius_sum_01_12,Plant_Biomass,Percent_flooding
3052,37.167147,-75.940768,2005,2005,0.380763,0.090994,-0.090350,-0.455445,0.177366,-0.033417,...,0.083835,0.164988,0.215945,0.103387,0.106825,0.078087,0.069956,63.683194,1003.28,0.796610
3053,37.167147,-75.940768,2016,2016,0.193519,0.041091,-0.073534,-0.263306,0.098206,0.008691,...,0.187380,0.281210,0.282970,0.188177,0.180670,0.146762,0.337905,54.088134,912.00,0.796610
3054,37.167148,-75.943672,2013,2013,0.335083,0.095114,0.086173,-0.256150,0.116894,0.028078,...,0.860322,1.267597,1.268477,0.695185,0.643402,0.518882,0.298263,233.109702,85.28,0.203980
3055,37.167154,-75.941307,2003,2003,0.513079,0.132720,0.034289,-0.487149,0.138752,-0.029422,...,0.087025,0.176565,0.253015,0.081277,0.086695,0.065410,0.004198,59.967806,468.80,0.164384
3056,37.167154,-75.941307,2004,2004,0.487674,0.122360,0.106857,-0.401753,0.303548,-0.063554,...,0.048710,0.091197,0.114132,0.039305,0.044640,0.023850,0.002542,26.225790,345.04,0.164384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3552,37.481998,-75.819360,2002,2002,0.407277,0.103597,-0.040103,-0.441163,0.135267,-0.003203,...,0.234530,0.471497,0.670103,0.289227,0.302592,0.211952,0.104006,131.053753,317.68,0.109272
3553,37.482109,-75.818885,1999,1999,0.568293,0.130879,0.100846,-0.495865,0.226073,-0.074702,...,0.034822,0.076925,0.103325,0.028442,0.033035,0.020852,0.000909,15.158241,226.40,0.050000
3554,37.482109,-75.818885,2000,2000,0.564525,0.160160,0.128746,-0.473543,0.189670,-0.008712,...,0.273442,0.539037,0.617990,0.242532,0.222513,0.177935,0.135329,87.070672,771.20,0.050000
3555,37.482293,-75.819265,1999,1999,0.519408,0.130932,0.150791,-0.399922,0.266252,-0.037644,...,0.192392,0.380602,0.453697,0.177405,0.182520,0.138795,0.027743,73.847030,104.00,0.133080


In [127]:
##Exporting coordinates

out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'GAVA_LANDSAT.csv')

#df_final.to_csv(out_csv)

In [123]:
##DF for GA_VA extraction

allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Combined_GA_VA/GAVA_coords.csv'

x = pd.read_csv(allplots_fc).reset_index()

fc_all = geemap.pandas_to_ee(x, latitude = "Latitude", longitude = "Longitude")

# Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)
# Map.addLayer(fc_all)
# Map



In [89]:
##Elevation (and z star and slope) extraction

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature


##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
slope = ee.Terrain.slope(dem)
dem_with_slope = dem.addBands(slope)

#Elevation
dem_extract = demExtraction(dem_with_slope)

dem_dict = dem_extract.getInfo()

# Extract the properties of the features into a list of dictionaries
features = [feature['properties'] for feature in dem_dict['features']]

# Convert the list of dictionaries into a pandas DataFrame
dem_vals = pd.DataFrame(features)
dem_vals

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,elevation,index,slope
0,31.308449,-81.415129,0.014085,71.520,2023,2023,1.179146,0,0.693467
1,31.308449,-81.415129,0.014085,140.192,2018,2018,1.179146,1,0.693467
2,31.308449,-81.415129,0.014085,161.872,2020,2020,1.179146,2,0.693467
3,31.308449,-81.415129,0.014085,170.976,2016,2016,1.179146,3,0.693467
4,31.308449,-81.415129,0.014085,250.240,2019,2019,1.179146,4,0.693467
...,...,...,...,...,...,...,...,...,...
4055,37.482109,-75.818885,0.050000,226.400,1999,1999,0.749837,4055,0.170882
4056,37.482109,-75.818885,0.050000,771.200,2000,2000,0.749837,4056,0.170882
4057,37.482293,-75.819265,0.133080,104.000,1999,1999,0.750699,4057,0.089179
4058,37.482293,-75.819265,0.133080,156.000,2002,2002,0.750699,4058,0.089179


In [90]:
years_dm = range(2000, 2023) ##range in which biomass data were collected

In [91]:
##For Daymet/gridmet images:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30,
        tileScale = 16 #ADDED 10/6/2022 - make sure it doesn't affect results (see thread below)
    )
    return feature

In [92]:
#Daymet
def MayOct_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

def MayOct_Max (collection, years):
  maxim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_maxim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year})
      maxim.append (Monthly_maxim)
  return ee.ImageCollection.fromImages(maxim)

def MayOct_Min (collection, years):
  minim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_minim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year})
      minim.append (Monthly_minim)
  return ee.ImageCollection.fromImages(minim)

def dm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def dm_earlyseason (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 5, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def peakdm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)



##COLLECTION
daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_all)

#Resampling daymet collection to 30m
def resamp_dm(img):
    resampled_dm = img.reproject(
        crs = 'EPSG:4326',
        scale = 30
    ).resample('bilinear')
    return resampled_dm
resamp_daymet = daymet.map(resamp_dm) #not sure if anything is actually changed

#Over year
monthly_dm = dm_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_list = monthly_dm.toList(monthly_dm.size())

#Peak biomass
peak_dm = peakdm_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
peakdm_list = peak_dm.toList(peak_dm.size())


#Early season
early_dm = dm_earlyseason(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
early_list = early_dm.toList(early_dm.size())

#Growing season
MayOct_dm = MayOct_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_MayOct_list = MayOct_dm.toList(MayOct_dm.size())

#Max
Max_dm = MayOct_Max(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_Max_list = Max_dm.toList(Max_dm.size())

#Min
Min_dm = MayOct_Min(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_Min_list = Min_dm.toList(Min_dm.size())

In [93]:
#Gridmet functions, pay attention to date
def gm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 1, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def gm_earlyseason (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 5, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 3, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def peakgm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 8, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def gmMayOct_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 5, 1).millis()})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

def gmMayOct_Max (collection, years):
  maxim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_maxim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 5, 1).millis()})
      maxim.append (Monthly_maxim)
  return ee.ImageCollection.fromImages(maxim)

def gmMayOct_Min (collection, years):
  minim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_minim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year, 'system:time_start': ee.Date.fromYMD(year, 5, 1).millis()})
      minim.append (Monthly_minim)
  return ee.ImageCollection.fromImages(minim)

gridmet = ee.ImageCollection('GRIDMET/DROUGHT').filterBounds(fc_all)

def add_year(image):
    year = ee.Date(image.get("system:time_start")).get("year")
    return image.set("Year", year)

gridmet_resamp = gridmet.map(resamp_dm).map(add_year)

#Over year
monthly_gridmet = gm_Avg(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_list = monthly_gridmet.toList(monthly_gridmet.size())

#Peak biomass
peak_gridmet = peakgm_Avg(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
peakgridmet_list = peak_gridmet.toList(peak_gridmet.size())

#Early season
early_gridmet = gm_earlyseason(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
earlygridmet_list = early_gridmet.toList(early_gridmet.size())

#Growing season
MayOct_gridmet = gmMayOct_Avg(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_MayOct_list = MayOct_gridmet.toList(MayOct_gridmet.size())

#Max
Max_gridmet = gmMayOct_Max(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_Max_list = Max_gridmet.toList(Max_gridmet.size())

#Min
Min_gridmet = gmMayOct_Min(gridmet_resamp, years = years_dm) #Updated to resampled daymet collection!
gridmet_Min_list = Min_gridmet.toList(Min_gridmet.size())

In [94]:
##Daymet for loop
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
daymet_extract = pd.concat(daymet_list)

daymet_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,43200.085938,106,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,43200.085938,138,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
162,31.334598,-81.478510,0.572337,893.676,2000,2000,43200.085938,162,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,43200.085938,176,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
206,31.334682,-81.478564,0.010638,961.730,2000,2000,43200.085938,206,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,43200.085938,2983,3.216712,335.934174,0.0,26.028383,14.624165,1825.712524
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,43200.085938,2992,3.216712,335.934174,0.0,26.028383,14.624165,1825.712524
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,43200.085938,3007,3.216712,335.934174,0.0,26.028383,14.624165,1825.712524
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,43200.085938,3024,3.218356,335.944458,0.0,26.017342,14.656713,1827.683594


In [108]:
print(min(daymet_extract['tmax']))
print(max(daymet_extract['tmax']))

18.81128692626953
26.589672088623047


In [109]:
#Daymet peak time
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(peakdm_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
# daymet_list = []
# for i in range(len(years_dm)):
#     dm_x = geemap.ee_to_pandas(rasterExtraction(ee.Image(peakdm_list.get(i))))
#     sampledm = dm_x[dm_x['Year'] == years_dm[i]]
#     daymet_list.append(sampledm) 
    
peakdaymet_extract = pd.concat(daymet_list)

peakdaymet_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,43638.546875,106,3.477500,347.249023,0,28.957935,19.154131,2298.904541
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,43638.546875,138,3.477500,347.249023,0,28.957935,19.154131,2298.904541
162,31.334598,-81.478510,0.572337,893.676,2000,2000,43638.546875,162,3.477500,347.249023,0,28.957935,19.154131,2298.904541
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,43638.546875,176,3.477500,347.249023,0,28.957935,19.154131,2298.904541
206,31.334682,-81.478564,0.010638,961.730,2000,2000,43638.546875,206,3.477500,347.249023,0,28.957935,19.154131,2298.904541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,43752.890625,2983,4.347609,348.728699,0,29.400652,18.574131,2237.086670
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,43752.890625,2992,4.347609,348.728699,0,29.400652,18.574131,2237.086670
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,43752.890625,3007,4.347609,348.728699,0,29.400652,18.574131,2237.086670
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,43752.890625,3024,4.428804,348.429901,0,29.387609,18.605543,2241.084229


In [110]:
#MayOct daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_MayOct_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_MayOct_extract = pd.concat(daymet_list)

dm_MayOct_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,46667.121094,106,2.749728,370.826019,0,30.362881,20.198153,2400.801758
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,46667.121094,138,2.749728,370.826019,0,30.362881,20.198153,2400.801758
162,31.334598,-81.478510,0.572337,893.676,2000,2000,46667.121094,162,2.749728,370.826019,0,30.362881,20.198153,2400.801758
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,46667.121094,176,2.749728,370.826019,0,30.362881,20.198153,2400.801758
206,31.334682,-81.478564,0.010638,961.730,2000,2000,46667.121094,206,2.749728,370.826019,0,30.362881,20.198153,2400.801758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,46748.886719,2983,4.351739,359.242950,0,30.804783,20.081739,2416.799316
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,46748.886719,2992,4.351739,359.242950,0,30.804783,20.081739,2416.799316
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,46748.886719,3007,4.351739,359.242950,0,30.804783,20.081739,2416.799316
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,46748.886719,3024,4.390380,359.041748,0,30.791359,20.110870,2419.565674


In [111]:
#Max daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_Max_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_Max_extract = pd.concat(daymet_list)

dm_Max_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,50542.179688,106,38.209999,509.709991,0,39.799999,25.530001,3253.010010
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,50542.179688,138,38.209999,509.709991,0,39.799999,25.530001,3253.010010
162,31.334598,-81.478510,0.572337,893.676,2000,2000,50542.179688,162,38.209999,509.709991,0,39.799999,25.530001,3253.010010
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,50542.179688,176,38.209999,509.709991,0,39.799999,25.530001,3253.010010
206,31.334682,-81.478564,0.010638,961.730,2000,2000,50542.179688,206,38.209999,509.709991,0,39.799999,25.530001,3253.010010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,50610.128906,2983,64.260002,538.280029,0,38.970001,25.020000,3171.030029
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,50610.128906,2992,64.260002,538.280029,0,38.970001,25.020000,3171.030029
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,50610.128906,3007,64.260002,538.280029,0,38.970001,25.020000,3171.030029
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,50610.128906,3024,65.639999,539.090027,0,38.970001,25.049999,3176.689941


In [112]:
#Min daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_Min_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_Min_extract = pd.concat(daymet_list)

dm_Min_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,50542.179688,106,38.209999,509.709991,0,39.799999,25.530001,3253.010010
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,50542.179688,138,38.209999,509.709991,0,39.799999,25.530001,3253.010010
162,31.334598,-81.478510,0.572337,893.676,2000,2000,50542.179688,162,38.209999,509.709991,0,39.799999,25.530001,3253.010010
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,50542.179688,176,38.209999,509.709991,0,39.799999,25.530001,3253.010010
206,31.334682,-81.478564,0.010638,961.730,2000,2000,50542.179688,206,38.209999,509.709991,0,39.799999,25.530001,3253.010010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,50610.128906,2983,64.260002,538.280029,0,38.970001,25.020000,3171.030029
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,50610.128906,2992,64.260002,538.280029,0,38.970001,25.020000,3171.030029
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,50610.128906,3007,64.260002,538.280029,0,38.970001,25.020000,3171.030029
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,50610.128906,3024,65.639999,539.090027,0,38.970001,25.049999,3176.689941


In [113]:
#Early daymet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(early_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
earlydaymet_extract = pd.concat(daymet_list)

earlydaymet_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,45918.375000,106,2.505326,396.561188,0.0,25.920870,14.480000,1712.289551
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,45918.375000,138,2.505326,396.561188,0.0,25.920870,14.480000,1712.289551
162,31.334598,-81.478510,0.572337,893.676,2000,2000,45918.375000,162,2.505326,396.561188,0.0,25.920870,14.480000,1712.289551
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,45918.375000,176,2.505326,396.561188,0.0,25.920870,14.480000,1712.289551
206,31.334682,-81.478564,0.010638,961.730,2000,2000,45918.375000,206,2.505326,396.561188,0.0,25.920870,14.480000,1712.289551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,45841.425781,2983,2.679456,387.008270,0.0,26.675434,14.459348,1711.481323
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,45841.425781,2992,2.679456,387.008270,0.0,26.675434,14.459348,1711.481323
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,45841.425781,3007,2.679456,387.008270,0.0,26.675434,14.459348,1711.481323
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,45841.425781,3024,2.656087,386.881195,0.0,26.661739,14.495109,1710.985718


In [114]:
#Gridmet season
years_dm = range(2000, 2023)

def add_year(image):
    return image.set("Year", ee.Date(image.date()).get("year"))

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_extract = pd.concat(daymet_list)


#Gridmet peak time
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(peakgridmet_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
peakgridmet_extract = pd.concat(daymet_list)



#MayOct gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_MayOct_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_MayOct_extract = pd.concat(daymet_list)

#Max gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_Max_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_Max_extract = pd.concat(daymet_list)

#Min gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(gridmet_Min_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
gridmet_Min_extract = pd.concat(daymet_list)

#Early gridmet extraction
years_dm = range(2000, 2023)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(add_year(ee.Image(earlygridmet_list.get(i))))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
earlygridmet_extract = pd.concat(daymet_list)

In [118]:
daymet_extract

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,swe,tmax,tmin,vp
106,31.334524,-81.478495,0.572337,974.992,2000,2000,43200.085938,106,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
138,31.334597,-81.478039,0.017241,2059.572,2000,2000,43200.085938,138,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
162,31.334598,-81.478510,0.572337,893.676,2000,2000,43200.085938,162,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
176,31.334600,-81.478147,0.014950,1291.824,2000,2000,43200.085938,176,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
206,31.334682,-81.478564,0.010638,961.730,2000,2000,43200.085938,206,2.525425,345.478119,0.0,25.100000,14.221671,1780.730225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,31.539429,-81.422410,0.020000,488.928,2022,2022,43200.085938,2983,3.216712,335.934174,0.0,26.028383,14.624165,1825.712524
2635,31.539457,-81.422250,0.011461,255.904,2022,2022,43200.085938,2992,3.216712,335.934174,0.0,26.028383,14.624165,1825.712524
2650,31.539480,-81.422098,0.011461,297.648,2022,2022,43200.085938,3007,3.216712,335.934174,0.0,26.028383,14.624165,1825.712524
2667,31.539491,-81.421750,0.011594,230.864,2022,2022,43200.085938,3024,3.218356,335.944458,0.0,26.017342,14.656713,1827.683594


In [121]:
#Final steps - extra rows in merge?

# landsat_extract = MayOct_extract ##NOT sure if this is the right way to do this - 
#where are all these dfs stored? (in streamlined folder in biomass datasets/sapelo)

#rolling_extract, earlydaymet_extract, early_extract (Landsat)

#daymet
first_extract = daymet_extract
dfx = pd.merge(daymet_extract, dem_vals, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how = 'left',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


# #dfx2 and daymet mayoct OR YEAR
# dfx3 = pd.merge(dfx2_c, daymet_extract, on = ['Plant_Biomass', 'Plot', 'Zone', 'Site', 'Year'], how='right')


#dfx3_a and daymet early (marMay)
earlydaymet_extract.rename(
    columns={
        'swe':'swe_daymMarMay','tmax':'tmax_daymMarMay','tmin':'tmin_daymMarMay','srad':'srad_daymMarMay', 'vp':'vp_daymMarMay',
        'prcp':'prcp_daymMarMay', 'dayl':'dayl_daymMarMay'
    }, inplace=True
)

dfx3_b = pd.merge(dfx, earlydaymet_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_b and daymet growing (MayOct)
dm_MayOct_extract.rename(
    columns={
        'swe':'swe_daymMayOct','tmax':'tmax_daymMayOct','tmin':'tmin_daymMayOct','srad':'srad_daymMayOct', 'vp':'vp_daymMayOct',
        'prcp':'prcp_daymMayOct', 'dayl':'dayl_daymMayOct'
    }, inplace=True
)

dfx3_c = pd.merge(dfx3_b, dm_MayOct_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx3_c and daymet max (MayOct)
dm_Max_extract.rename(
    columns={
        'swe':'swe_Max','tmax':'tmax_Max','tmin':'tmin_Max','srad':'srad_Max', 'vp':'vp_Max',
        'prcp':'prcp_Max', 'dayl':'dayl_Max'
    }, inplace=True
)

dfx3_d = pd.merge(dfx3_c, dm_Max_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx3_d and daymet min (MayOct)
dm_Min_extract.rename(
    columns={
        'swe':'swe_Min','tmax':'tmax_Min','tmin':'tmin_Min','srad':'srad_Min', 'vp':'vp_Min',
        'prcp':'prcp_Min', 'dayl':'dayl_Min'
    }, inplace=True
)

dfx3_e = pd.merge(dfx3_d, dm_Min_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_e and daymet peak
peakdaymet_extract.rename(
    columns={
        'swe':'swe_peak','tmax':'tmax_peak','tmin':'tmin_peak','srad':'srad_peak', 'vp':'vp_peak',
        'prcp':'prcp_peak', 'dayl':'dayl_peak'
    }, inplace=True
)

dfx3_f = pd.merge(dfx3_e, peakdaymet_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#GRIDMET ADDED
#dfx3_f and gridmet mayoct OR YEAR
dfx4_a = pd.merge(dfx3_f, gridmet_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
                 suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx4_a and gridmet early (marMay)
earlygridmet_extract.rename(
    columns={
        'spei30d':'spei30d_MarMay','spei90d':'spei90d_MarMay','spei180d':'spei180d_MarMay','spei1y':'spei1y_MarMay', 
        'spei5y':'spei5y_MarMay',
        'pdsi':'pdsi_MarMay', 'pz':'pz_MarMay'
    }, inplace=True
)

dfx4_c = pd.merge(dfx4_a, earlygridmet_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx4_c and gridmet growing (MayOct)
gridmet_MayOct_extract.rename(
    columns={
        'spei30d':'spei30d_MayOct','spei90d':'spei90d_MayOct','spei180d':'spei180d_MayOct','spei1y':'spei1y_MayOct', 
        'spei5y':'spei5y_MayOct',
        'pdsi':'pdsi_MayOct', 'pz':'pz_MayOct'
    }, inplace=True
)

dfx4_d = pd.merge(dfx4_c, gridmet_MayOct_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx4_d and gridmet max (MayOct)
gridmet_Max_extract.rename(
    columns={
        'spei30d':'spei30d_Max','spei90d':'spei90d_Max','spei180d':'spei180d_Max','spei1y':'spei1y_Max', 
        'spei5y':'spei5y_Max',
        'pdsi':'pdsi_Max', 'pz':'pz_Max'
    }, inplace=True
)

dfx4_e = pd.merge(dfx4_d, gridmet_Max_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx4_e and gridmet min (MayOct)
gridmet_Min_extract.rename(
    columns={
        'spei30d':'spei30d_Min','spei90d':'spei90d_Min','spei180d':'spei180d_Min','spei1y':'spei1y_Min', 
        'spei5y':'spei5y_Min',
        'pdsi':'pdsi_Min', 'pz':'pz_Min'
    }, inplace=True
)

dfx4_f = pd.merge(dfx4_e, gridmet_Min_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx4_f and gridmet peak
peakgridmet_extract.rename(
    columns={
        'spei30d':'spei30d_gridmet','spei90d':'spei90d_gridmet','spei180d':'spei180d_gridmet','spei1y':'spei1y_gridmet', 
        'spei5y':'spei5y_gridmet',
        'pdsi':'pdsi_gridmet', 'pz':'pz_gridmet'
    }, inplace=True
)

df = pd.merge(dfx4_f, peakgridmet_extract, on = ['Plant_Biomass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

display(df)

,Latitude,Longitude,Percent_flooding,Plant_Biomass,Sampling_year,Year,dayl,index,prcp,srad,...,spei1y_Min,spei30d_Min,spei5y_Min,spei90d_Min,pdsi_gridmet,spei180d_gridmet,spei1y_gridmet,spei30d_gridmet,spei5y_gridmet,spei90d_gridmet
0,31.334524,-81.478495,0.572337,974.992,2000.0,2000,43200.085938,106.0,2.525425,345.478119,...,-0.39,1.69,0.47,0.47,-2.891579,-0.688947,-1.158947,-0.023158,-0.135263,-0.309474
1,31.334597,-81.478039,0.017241,2059.572,2000.0,2000,43200.085938,138.0,2.525425,345.478119,...,-0.39,1.69,0.47,0.47,-2.891579,-0.688947,-1.158947,-0.023158,-0.135263,-0.309474
2,31.334598,-81.478510,0.572337,893.676,2000.0,2000,43200.085938,162.0,2.525425,345.478119,...,-0.39,1.69,0.47,0.47,-2.891579,-0.688947,-1.158947,-0.023158,-0.135263,-0.309474
3,31.334600,-81.478147,0.014950,1291.824,2000.0,2000,43200.085938,176.0,2.525425,345.478119,...,-0.39,1.69,0.47,0.47,-2.891579,-0.688947,-1.158947,-0.023158,-0.135263,-0.309474
4,31.334682,-81.478564,0.010638,961.730,2000.0,2000,43200.085938,206.0,2.525425,345.478119,...,-0.39,1.69,0.47,0.47,-2.891579,-0.688947,-1.158947,-0.023158,-0.135263,-0.309474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,31.539429,-81.422410,0.020000,488.928,2022.0,2022,43200.085938,2983.0,3.216712,335.934174,...,0.47,1.69,1.01,1.69,0.091111,0.268889,-0.154444,0.044444,0.879444,0.567222
3796,31.539457,-81.422250,0.011461,255.904,2022.0,2022,43200.085938,2992.0,3.216712,335.934174,...,0.47,1.69,1.01,1.69,0.091111,0.268889,-0.154444,0.044444,0.879444,0.567222
3797,31.539480,-81.422098,0.011461,297.648,2022.0,2022,43200.085938,3007.0,3.216712,335.934174,...,0.47,1.69,1.01,1.69,0.091111,0.268889,-0.154444,0.044444,0.879444,0.567222
3798,31.539491,-81.421750,0.011594,230.864,2022.0,2022,43200.085938,3024.0,3.218356,335.944458,...,0.47,1.69,1.01,1.69,0.091111,0.268889,-0.154444,0.044444,0.879444,0.567222


In [124]:
for col in daymet_extract.columns:
    print(col)
    
##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'gava_dm_gm_elev.csv')
# df.to_csv(out_csv, index = False)

Latitude
Longitude
Percent_flooding
Plant_Biomass
Sampling_year
Year
dayl
index
prcp
srad
swe
tmax
tmin
vp


array([31.334524  , 31.334597  , 31.334598  , 31.3346    , 31.334682  ,
       31.334745  , 31.334841  , 31.334858  , 31.334934  , 31.334943  ,
       31.335013  , 31.335086  , 31.33509   , 31.335155  , 31.351153  ,
       31.351196  , 31.351197  , 31.387539  , 31.387549  , 31.38764   ,
       31.387745  , 31.387748  , 31.387844  , 31.38786   , 31.387957  ,
       31.388295  , 31.388423  , 31.435437  , 31.435543  , 31.435647  ,
       31.435748  , 31.435795  , 31.43583   , 31.435893  , 31.4359    ,
       31.435959  , 31.43597   , 31.435977  , 31.436007  , 31.436011  ,
       31.43609   , 31.451197  , 31.451281  , 31.451372  , 31.451373  ,
       31.451473  , 31.451478  , 31.451548  , 31.451676  , 31.451768  ,
       31.451893  , 31.452232  , 31.452363  , 31.452485  , 31.452632  ,
       31.476455  , 31.476504  , 31.476595  , 31.476614  , 31.47673   ,
       31.476808  , 31.476972  , 31.476985  , 31.476992  , 31.477007  ,
       31.477098  , 31.477262  , 31.518161  , 31.518198  , 31.51